# ODS to 12Twenty - Python Version

## Overview
This program uses python, the HTTP requests library, "requests", and the data analysis/manipulation library, "pandas".  It is currently in the form of a Jupyter Notebook.  The notebook approach is very handy in development since you can run "cells" one at a time and check the data as you go.  In production, this code would run as a simple python script.  Currently, it uses a static version of the ODS data, imported to our DEV SQL Server via SSIS.  However, there is no reason that the python script couldn't just query ODS directly, eliminating the need for SSIS. 

In [299]:
#install and import required libraries
!pip3 install requests
!pip3 install pyodbc
import json, requests, pandas as pd, pyodbc

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [300]:
#authenticate with 12Twenty
base_url = 'https://siu.admin.12twenty.com/api'
auth_key = open('api_key.txt', 'r').read()
auth_url = base_url + '/client/generateAuthenticationToken?key=' + auth_key

bearer_token = requests.get(auth_url).text
#set authorization header for future requests
auth_header = {'Authorization': 'Bearer ' + bearer_token}

In [313]:
#pull first page of json students from 12Twenty
students_url = base_url + '/V2/students'
students_params = {'PageSize' : '500'}
twelve_twenty_students_json = requests.get(students_url, headers=auth_header, params=students_params)
#print(json.dumps(twelve_twenty_students_json.json(), indent=2))

In [ ]:
#convert json to a pandas dataframe
twelve_twenty_students_json = json.loads(twelve_twenty_students_json.text)
twelve_twenty_students_page = pd.json_normalize(twelve_twenty_students_json['Items'])
twelve_twenty_students_page.head()

In [303]:
#create array to store paginated 12Twenty data
twelve_twenty_students = []
twelve_twenty_students.append(twelve_twenty_students_page)
#iterate through all pages of 12Twenty students and assemble array of dataframes
num_pages = twelve_twenty_students_json['NumberOfPages']
for page in range(2, num_pages + 1):
    students_params = {'PageSize' : '500', 'PageNumber': page}
    twelve_twenty_students_json = json.loads(requests.get(students_url, headers=auth_header, params=students_params).text)
    twelve_twenty_students_page = pd.json_normalize(twelve_twenty_students_json['Items'])
    twelve_twenty_students.append(twelve_twenty_students_page)

In [ ]:
#convert array of data frames into one master data frame
twelve_twenty_students = pd.concat(twelve_twenty_students)

print(twelve_twenty_students.count())

In [ ]:
#remove unnecessary columns and rename remaining ones to match ODS columns

#print(twelve_twenty_students.columns.tolist())
columnMapping = {'Id':'12TwentyId',
                 'FirstName':'FirstName',
                 'MiddleName':'MiddleName',
                 'LastName' : 'LastName',
                 'EmailAddress' : 'EmailAddress',
                 'IsEnrolled': 'IsEnrolled',
                 'StudentId': 'StudentId',
                 'College.Name': 'College',
                 'Program.Name': 'Program',
                 'GraduationYearId' : 'GraduationYear',
                 'GraduationTerm' : 'GraduationTerm',
                 'DegreeLevel.Name' : 'DegreeLevel',
                 'CustomAttributeValues.custom_attribute_10888805132042': 'AppliedForGraduation'
}

for column in twelve_twenty_students.columns:
    if column not in columnMapping.keys():
        twelve_twenty_students.drop(column, axis=1, inplace=True)

twelve_twenty_students.rename(columns = columnMapping, inplace='True')

#set primary key
twelve_twenty_students.set_index('StudentId')

twelve_twenty_students.head()


In [306]:
#set up connection to SQL db
sql_connection = pyodbc.connect("Driver={SQL Server};"
                      "Server=itapp-ssis-dev;"
                      "Database=MableySandbox;"
                      "Trusted_Connection=yes;")


In [ ]:
#query ODS
ods_query = "SELECT * FROM TwelveTwentyStudents;"
ods_students = pd.read_sql(ods_query,sql_connection)
#add action and reporting columns
ods_students.insert(0, 'ActionNeeded', 'None')
ods_students.insert(len(ods_students.columns), 'Result', '')
ods_students.insert(len(ods_students.columns), 'Message', '')
#strip year from graduation term
ods_students['GraduationTerm'] = ods_students['GraduationTerm'].str.replace('[^a-zA-Z]*', '', regex=True)
#set primary key
ods_students.set_index('StudentId')

In [ ]:
#if ODS student ID is not found in 12Twenty list, mark it for Insert
ods_students.loc[~ods_students['StudentId'].isin(twelve_twenty_students['StudentId']), 'ActionNeeded'] = 'Insert'
ods_students.loc[ods_students['ActionNeeded'] == 'Insert']

In [309]:
#left join ODS and 12Twenty dataframes with suffixes for data origin on column names
merged_students = pd.merge(ods_students, twelve_twenty_students, on ='StudentId', how='left', suffixes=['_ODS', '_12Twenty'])
merged_students.head()

,ActionNeeded,EmailAddress_ODS,FirstName_ODS,MiddleName_ODS,LastName_ODS,StudentId,GraduationYear_ODS,GraduationTerm_ODS,ExpectedGraduationDate,IsEnrolled_ODS,...,EmailAddress_12Twenty,GraduationYear_12Twenty,GraduationTerm_12Twenty,12TwentyId,IsEnrolled_12Twenty,College_12Twenty,Program_12Twenty,DegreeLevel_12Twenty,AppliedForGraduation,MiddleName_12Twenty
0,None,danielle.cox@siu.edu,Danielle,Nichole,Cox,SIU850000649,2025,Spring,2025-05-09,Yes,...,danielle.cox@siu.edu,2025.0,Spring,5.400161e+14,True,SIU,SIU,Senior,True,Nichole
1,None,james.w.black@siu.edu,James,W,Black,SIU850001358,2025,Spring,2025-05-09,Yes,...,james.w.black@siu.edu,2025.0,Spring,5.400161e+14,True,SIU,SIU,Senior,False,W
2,None,thomas.distelhurst@siu.edu,Thomas,None,Distelhurst,SIU850001815,2024,Spring,2024-05-10,Yes,...,thomas.distelhurst@siu.edu,2024.0,Spring,5.400161e+14,True,College of Arts and Media,SIU,Alumni,False,NaN
3,None,mnowak@siu.edu,Matthew,G,Nowak,SIU850001876,2024,Summer,2024-08-02,Yes,...,mnowak@siu.edu,2024.0,Summer,5.400161e+14,True,College of Liberal Arts,SIU,Alumni,False,G
4,None,vickie.walls@siu.edu,Vickie,Anna,McKinney,SIU850002168,2025,Spring,2025-05-09,Yes,...,vickie.walls@siu.edu,2025.0,Spring,5.400161e+14,True,SIU,SIU,Alumni,False,Anna


In [310]:
#mark existing 12Twenty students for update where data differs from ODS to 12Twenty
merged_students.loc[(merged_students['ActionNeeded'] != 'Insert') &
                    (
                    (merged_students['EmailAddress_ODS'] != merged_students['EmailAddress_12Twenty']) |
                    (merged_students['FirstName_ODS']!= merged_students['FirstName_12Twenty']) |
                    (merged_students['LastName_ODS']!= merged_students['LastName_12Twenty']) |
                    (merged_students['DegreeLevel_ODS']!= merged_students['DegreeLevel_12Twenty']) |
                    (merged_students['GraduationYear_ODS']!= merged_students['GraduationYear_12Twenty']) |
                    (merged_students['GraduationTerm_ODS']!= merged_students['GraduationTerm_12Twenty']) |
                    (merged_students['AppliedForGrad'] != merged_students['AppliedForGraduation']) 
                    ), 'ActionNeeded'] = 'Update'
merged_students.loc[merged_students['ActionNeeded'] == 'Update'].head()


,ActionNeeded,EmailAddress_ODS,FirstName_ODS,MiddleName_ODS,LastName_ODS,StudentId,GraduationYear_ODS,GraduationTerm_ODS,ExpectedGraduationDate,IsEnrolled_ODS,...,EmailAddress_12Twenty,GraduationYear_12Twenty,GraduationTerm_12Twenty,12TwentyId,IsEnrolled_12Twenty,College_12Twenty,Program_12Twenty,DegreeLevel_12Twenty,AppliedForGraduation,MiddleName_12Twenty
566,Update,clinton.huse@hotmail.com,Clinton,Dwayne,Huse,SIU850444237,2010,Spring,2010-05-14,Yes,...,clinton.huse@hotmail.com,2028.0,Summer,5.400161e+14,True,SIU,SIU,Senior,False,Dwayne
867,Update,ivan.vargas@siu.edu,Ivanovich,None,Vargas,SIU850901961,2026,Spring,2026-05-08,Yes,...,ivan.vargas@siu.edu,2026.0,Spring,5.400161e+14,True,College of Health and Human Sciences,SIU,Senior,False,NaN
2642,Update,jazzmine.crane@siu.edu,Johnathan,None,Crane,SIU854221480,2025,Spring,2025-05-09,Yes,...,jazzmine.crane@siu.edu,2025.0,Spring,5.400161e+14,True,SIU,SIU,Senior,False,NaN
2797,Update,nicole.lach@siu.edu,Nicole,A,Lach,SIU854339107,2024,None,2024-12-14,Yes,...,nicole.lach@siu.edu,2024.0,Spring,5.400161e+14,True,SIU,SIU,Senior,True,A
2863,Update,holton.zoss@siu.edu,Holton,A,Zoss,SIU854365285,2024,None,2024-12-14,Yes,...,holton.zoss@siu.edu,2024.0,Spring,5.400161e+14,True,SIU,SIU,Senior,True,A


In [314]:
#display final counts and data table
print(f"Students to Insert: {merged_students.loc[merged_students['ActionNeeded'] == 'Insert']['StudentId'].count()}") 
print(f"Students to Update: {merged_students.loc[merged_students['ActionNeeded'] == 'Update']['StudentId'].count()}") 
with pd.option_context('display.max_rows', 25, 'display.max_columns', None): 
    display(merged_students.loc[merged_students['ActionNeeded'] != 'None'].sort_values(by=['ActionNeeded', 'LastName_ODS', 'FirstName_ODS'], ascending=[False, True, True]))

Students to Insert: 393
Students to Update: 14


,ActionNeeded,EmailAddress_ODS,FirstName_ODS,MiddleName_ODS,LastName_ODS,StudentId,GraduationYear_ODS,GraduationTerm_ODS,ExpectedGraduationDate,IsEnrolled_ODS,Program_ODS,DegreeLevel_ODS,College_ODS,AppliedForGrad,SystemLabelNames,Result,Message,FirstName_12Twenty,LastName_12Twenty,EmailAddress_12Twenty,GraduationYear_12Twenty,GraduationTerm_12Twenty,12TwentyId,IsEnrolled_12Twenty,College_12Twenty,Program_12Twenty,DegreeLevel_12Twenty,AppliedForGraduation,MiddleName_12Twenty
3468,Update,nathaniel.conroy@siu.edu,Nathaniel,M,Conroy,SIU854760834,2024,None,2024-12-14,Yes,SIU,Senior,SIU,True,None,,,Nathaniel,Conroy,nathaniel.conroy@siu.edu,2024.0,Spring,5.400161e+14,True,SIU,SIU,Senior,True,M
2642,Update,jazzmine.crane@siu.edu,Johnathan,None,Crane,SIU854221480,2025,Spring,2025-05-09,Yes,SIU,Senior,SIU,False,None,,,Johnathan,Crane,jazzmine.crane@siu.edu,2025.0,Spring,5.400161e+14,True,SIU,SIU,Senior,False,NaN
13222,Update,kennedy.furano@siu.edu,Kennedy,None,Furano,SIU856570672,2028,Spring,2028-05-05,Yes,SIU,Sophomore,SIU,False,None,,,Kennedy,Furano,kennedy.furano@siu.edu,2028.0,Spring,5.400161e+14,True,SIU,SIU,Sophomore,False,NaN
2910,Update,smnazmul.haque@siu.edu,Nazmul,None,Haque,SIU854410428,2024,Spring,2024-05-10,Yes,SIU,Alumni,SIU,False,None,,,Nazmul,Haque,smnazmul.haque@siu.edu,2024.0,Spring,5.400161e+14,True,College of Health and Human Sciences,SIU,Alumni,False,NaN
566,Update,clinton.huse@hotmail.com,Clinton,Dwayne,Huse,SIU850444237,2010,Spring,2010-05-14,Yes,SIU,Alumni,SIU,False,None,,,Clinton,Huse,clinton.huse@hotmail.com,2028.0,Summer,5.400161e+14,True,SIU,SIU,Senior,False,Dwayne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,Insert,iseult.quirke@siu.edu,Iseult,Quirke,Young,SIU850327470,2024,Spring,2024-05-10,Yes,SIU,Alumni,SIU,False,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,Insert,teke4@siu.edu,Tequila,Anne,Young,SIU850033580,2023,Summer,2023-08-04,Yes,SIU,Alumni,SIU,False,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3094,Insert,samuel.zemen@siu.edu,Samuel,G,Zemen,SIU854444828,2024,Spring,2024-05-10,Yes,SIU,Alumni,SIU,False,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6848,Insert,haobo.zhao@siu.edu,Haobo,None,Zhao,SIU856498579,2023,Summer,2023-08-04,Yes,SIU,Alumni,SIU,False,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pushing Data to 12Twenty

From here, we would simply filter the merged list of students and use requests to POST/PATCH the students to insert/update back to 12Twenty using the _ODS columns.

## Logging/Reporting

We would log the result and any error message to the "Result" and "Message" columns of the dataframe, then use it to generate a nice HTML table for email reporting.  We could generate text logs, but there are also [python libraries for Splunk reporting](https://dev.splunk.com/enterprise/docs/devtools/python/sdk-python/).
